In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report, f1_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from keras import backend as K
from keras.models import model_from_json
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/UIT-ViFSD_me/Train.csv')
dev = pd.read_csv('/content/drive/MyDrive/UIT-ViFSD_me/Dev.csv')
test = pd.read_csv('/content/drive/MyDrive/UIT-ViFSD_me/Test.csv')

###process


In [ ]:
!wget https://gist.githubusercontent.com/nguyenvanhieuvn/7d9441c10b3c2739499fc5a4d9ea06fb/raw/df939245b3e841b62af115be4dcb3516dadc9fc5/teencode.txt

--2023-07-06 06:16:19--  https://gist.githubusercontent.com/nguyenvanhieuvn/7d9441c10b3c2739499fc5a4d9ea06fb/raw/df939245b3e841b62af115be4dcb3516dadc9fc5/teencode.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5656 (5.5K) [text/plain]
Saving to: ‘teencode.txt.1’

teencode.txt.1      100%[===================>]   5.52K  --.-KB/s    in 0s      

2023-07-06 06:16:19 (70.7 MB/s) - ‘teencode.txt.1’ saved [5656/5656]



In [ ]:
!pip install emoji --upgrade

In [ ]:
!pip install pyvi
!pip install -q vncorenlp
!git clone https://github.com/vncorenlp/VnCoreNLP.git

fatal: destination path 'VnCoreNLP' already exists and is not an empty directory.


In [ ]:
import re
import requests
import emoji
from pyvi import ViTokenizer

In [ ]:
#Tải danh sách stop words tiếng việt từ github
stop_words_url_1 = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt'
stop_words_data_1 = requests.get(stop_words_url_1).content.decode('utf-8')
stop_words_1 = stop_words_data_1.splitlines()

stop_words_url_2 = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt'
stop_words_data_2 = requests.get(stop_words_url_2).content.decode('utf-8')
stop_words_2 = stop_words_data_2.splitlines()

In [ ]:
# kết hợp các danh sách stop words có dấu
stop_words = list(set(stop_words_1 + stop_words_2))

In [ ]:
# định nghĩa hàm để loại bỏ các stop words
def remove_stop_words(words):
    return [word for word in words if word not in stop_words]

# định nghĩa hàm để tách câu thành các từ và loại bỏ dấu
def tokenize(text):
    return ViTokenizer.tokenize(text).lower().split()

In [ ]:
replace_list = {
    'ô kêi': 'ok', 'okie': 'ok', 'o kê': 'ok', 'okey': 'ok', 'ôkê': 'ok', 'oki': 'ok', 'oke': 'ok', 'okay': 'ok', 'okê': 'ok',
    'tks': 'cảm ơn', 'thks': 'cảm ơn', 'thanks': 'cảm ơn', 'ths': 'cảm ơn', 'thank': 'cảm ơn',
    'kg': 'không', 'not': 'không', 'k': 'không', 'kh': 'không', 'kô': 'không', 'hok': 'không', 'ko': 'không', 'khong': 'không', 'kp': 'không phải',
    'he he': 'tích cực', 'hehe': 'tích cực', 'hihi': 'tích cực', 'haha': 'tích cực', 'hjhj': 'tích cực', 'thick': 'tích cực',
    'lol': 'tiêu cực', 'cc': 'tiêu cực', 'huhu': 'tiêu cực', 'cute': 'dễ thương',

    'sz': 'cỡ', 'size': 'cỡ',
    'wa': 'quá', 'wá': 'quá', 'qá': 'quá',
    'đx': 'được', 'dk': 'được', 'dc': 'được', 'đk': 'được', 'đc': 'được',
    'vs': 'với', 'j': 'gì', '“': ' ', 'time': 'thời gian', 'm': 'mình', 'mik': 'mình', 'r': 'rồi', 'bjo': 'bao giờ', 'very': 'rất',

    'authentic': 'chuẩn chính hãng', 'aut': 'chuẩn chính hãng', 'auth': 'chuẩn chính hãng', 'date': 'hạn sử dụng', 'hsd': 'hạn sử dụng',
    'store': 'cửa hàng', 'sop': 'cửa hàng', 'shopE': 'cửa hàng', 'shop': 'cửa hàng',
    'sp': 'sản phẩm', 'product': 'sản phẩm', 'hàg': 'hàng',
    'ship': 'giao hàng', 'delivery': 'giao hàng', 'síp': 'giao hàng', 'order': 'đặt hàng',

    'gud': 'tốt', 'wel done': 'tốt', 'good': 'tốt', 'gút': 'tốt', 'tot': 'tốt', 'nice': 'tốt', 'perfect': 'rất tốt',
    'quality': 'chất lượng', 'chất lg': 'chất lượng', 'chat': 'chất', 'excelent': 'hoàn hảo', 'bt': 'bình thường',
    'sad': 'tệ', 'por': 'tệ', 'poor': 'tệ', 'bad': 'tệ',
    'beautiful': 'đẹp tuyệt vời', 'dep': 'đẹp',
    'xau': 'xấu', 'sấu': 'xấu',

    'thik': 'thích', 'iu': 'yêu', 'fake': 'giả mạo',
    'quickly': 'nhanh', 'quick': 'nhanh', 'fast': 'nhanh',
    'fresh': 'tươi', 'delicious': 'ngon',

    'dt': 'điện thoại', 'fb': 'facebook', 'face': 'facebook', 'ks': 'khách sạn', 'nv': 'nhân viên',
    'nt': 'nhắn tin', 'ib': 'nhắn tin', 'tl': 'trả lời', 'trl': 'trả lời', 'rep': 'trả lời',
    'fback': 'feedback', 'fedback': 'feedback',
    'sd': 'sử dụng', 'sài': 'xài',

    '^_^': 'tích cực', ':)': 'tích cực', ':(': 'tiêu cực',
    '❤️': 'tích cực', '👍': 'tích cực', '🎉': 'tích cực', '😀': 'tích cực', '😍': 'tích cực', '😂': 'tích cực', '🤗': 'tích cực', '😙': 'tích cực', '🙂': 'tích cực',
    '😔': 'tiêu cực', '😓': 'tiêu cực',
    '⭐': 'star', '*': 'star', '🌟': 'star',
}

with open('teencode.txt', encoding='utf-8') as f:
    for pair in f.readlines():
        key, value = pair.split('\t')
        replace_list[key] = value.strip()


def normalize_acronyms(text):
    words = []
    for word in text.strip().split():
        # word = word.strip(string.punctuation)
        if word.lower() not in replace_list.keys(): words.append(word)
        else: words.append(replace_list[word.lower()])
    return emoji.demojize(' '.join(words))

In [ ]:
def remove_HTML(text):
    return re.sub(r'<[^>]*>', '', text)

In [ ]:
# Standardize unicode
def convert_unicode(text):
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    char1252 = char1252.split('|')
    charutf8 = charutf8.split('|')

    dic = {}
    for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dic[x.group()], text
    )

In [ ]:
# Standardize accent typing
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)


def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True

In [ ]:
def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)


def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

In [ ]:
from vncorenlp import VnCoreNLP
from nltk import flatten

In [ ]:
# # Word segmentation
# annotator = VnCoreNLP('/content/VnCoreNLP/VnCoreNLP-1.1.1.jar')
# def word_segmentation(text):
#     words = annotator.tokenize(text)
#     return ' '.join(word for word in flatten(words))


# # Remove unnecessary characters
# def remove_unnecessary_characters(text):
#     text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ_]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
#     return text

In [ ]:
# def text_process(text):
#     text = remove_HTML(text)
#     text = convert_unicode(text)
#     # text = standardize_sentence_typing(text)
#     text = normalize_acronyms(text)
#     text = word_segmentation(text) # When use PhoBERT
#     text = remove_unnecessary_characters(text)
#     # return text.lower()
#     return text

In [ ]:
def text_process(train):
    processed_text = re.sub(r'\W', ' ', train)  # Ví dụ: loại bỏ ký tự không phải chữ cái và số
    processed_text = processed_text.lower()# Ví dụ: chuyển thành chữ thường
    a = normalize_acronyms(processed_text)
    b = tokenize(a)
    c = remove_stop_words(b)
    d = ' '.join(c)
    return d

In [ ]:
aa=['SCREEN','CAMERA','FEATURES','BATTERY','PERFORMANCE','STORAGE','DESIGN','PRICE','GENERAL','SER&ACC']
a =['SCREEN','CAMERA','FEATURES','BATTERY','PERFORMANCE','STORAGE','DESIGN','PRICE','GENERAL','SER&ACC','OTHERS']
s=['Positive','Negative','Neutral']
label=[]
aspect=[]
for i in range(0,10):
    x=[]
    for j in range(0,3):
        x.append(aa[i]+'#'+s[j])
    aspect.append(x)
for i in range (0,10):
    for j in range(0,3):
        label.append(aa[i]+'#'+s[j])
slabel=label.copy()
saspect=aspect.copy()
label.append('OTHERS')
aspect.append(['OTHERS'])

In [ ]:
def ag_matrix(df_name,r1,r2,c1=5,c2=37,l=label):
    df=df_name.copy()
    for i in range(0,len(l)):
        df[l[i]]=0
    for i in range(r1,r2):
        for j in range(0,len(l)):
            if l[j] in str(df['label'][i]):
                df[l[j]][i]=1
    m=df.iloc[r1:r2,c1:c2]
    return m

In [ ]:
dd_train=ag_matrix(train,0,len(train))
dd_test = ag_matrix(test,0,len(test))

<ipython-input-20-8f5f6da7b80e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[l[j]][i]=1
<ipython-input-20-8f5f6da7b80e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[l[j]][i]=1


In [ ]:
X_train = train['comment'].apply(text_process)
X_test = test['comment'].apply(text_process)

In [ ]:
Y_train = dd_train.astype('int')
Y_test = dd_test.astype('int')

#Naive Bayes

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer

In [ ]:
predictions = pd.DataFrame()
nb = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

for category in Y_train.columns:
    print('... Processing {}'.format(category))
    nb.fit(X_train, Y_train[category])
    prediction = nb.predict(X_test)
    predictions[category] = prediction

print(classification_report(Y_test, predictions, target_names=Y_test.columns, digits=4))

... Processing SCREEN#Positive
... Processing SCREEN#Negative
... Processing SCREEN#Neutral
... Processing CAMERA#Positive
... Processing CAMERA#Negative
... Processing CAMERA#Neutral
... Processing FEATURES#Positive
... Processing FEATURES#Negative
... Processing FEATURES#Neutral
... Processing BATTERY#Positive
... Processing BATTERY#Negative
... Processing BATTERY#Neutral
... Processing PERFORMANCE#Positive
... Processing PERFORMANCE#Negative
... Processing PERFORMANCE#Neutral
... Processing STORAGE#Positive
... Processing STORAGE#Negative
... Processing STORAGE#Neutral
... Processing DESIGN#Positive
... Processing DESIGN#Negative
... Processing DESIGN#Neutral
... Processing PRICE#Positive
... Processing PRICE#Negative
... Processing PRICE#Neutral
... Processing GENERAL#Positive
... Processing GENERAL#Negative
... Processing GENERAL#Neutral
... Processing SER&ACC#Positive
... Processing SER&ACC#Negative
... Processing SER&ACC#Neutral
... Processing OTHERS
                      precis

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#áp dụng preprocess


In [ ]:
predictions = pd.DataFrame()
nb = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

for category in Y_train.columns:
    print('... Processing {}'.format(category))
    nb.fit(X_train, Y_train[category])
    prediction = nb.predict(X_test)
    predictions[category] = prediction

print(classification_report(Y_test, predictions, target_names=Y_test.columns, digits=4))

... Processing SCREEN#Positive
... Processing SCREEN#Negative
... Processing SCREEN#Neutral
... Processing CAMERA#Positive
... Processing CAMERA#Negative
... Processing CAMERA#Neutral
... Processing FEATURES#Positive
... Processing FEATURES#Negative
... Processing FEATURES#Neutral
... Processing BATTERY#Positive
... Processing BATTERY#Negative
... Processing BATTERY#Neutral
... Processing PERFORMANCE#Positive
... Processing PERFORMANCE#Negative
... Processing PERFORMANCE#Neutral
... Processing STORAGE#Positive
... Processing STORAGE#Negative
... Processing STORAGE#Neutral
... Processing DESIGN#Positive
... Processing DESIGN#Negative
... Processing DESIGN#Neutral
... Processing PRICE#Positive
... Processing PRICE#Negative
... Processing PRICE#Neutral
... Processing GENERAL#Positive
... Processing GENERAL#Negative
... Processing GENERAL#Neutral
... Processing SER&ACC#Positive
... Processing SER&ACC#Negative
... Processing SER&ACC#Neutral
... Processing OTHERS
                      precis

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#linear SVC aspect_senti preprocessing

In [ ]:
from sklearn.svm import SVC
predictions = pd.DataFrame()
nb = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SVC(gamma='auto')),])
for category in Y_train.columns:
  print('... Processing {}'.format(category))
  nb.fit(X_train,Y_train[category])
  prediction = nb.predict(X_test)
  predictions[category] = prediction
print(classification_report(Y_test, predictions, target_names=Y_test.columns, digits= 4))

... Processing SCREEN#Positive
... Processing SCREEN#Negative
... Processing SCREEN#Neutral
... Processing CAMERA#Positive
... Processing CAMERA#Negative
... Processing CAMERA#Neutral
... Processing FEATURES#Positive
... Processing FEATURES#Negative
... Processing FEATURES#Neutral
... Processing BATTERY#Positive
... Processing BATTERY#Negative
... Processing BATTERY#Neutral
... Processing PERFORMANCE#Positive
... Processing PERFORMANCE#Negative
... Processing PERFORMANCE#Neutral
... Processing STORAGE#Positive
... Processing STORAGE#Negative
... Processing STORAGE#Neutral
... Processing DESIGN#Positive
... Processing DESIGN#Negative
... Processing DESIGN#Neutral
... Processing PRICE#Positive
... Processing PRICE#Negative
... Processing PRICE#Neutral
... Processing GENERAL#Positive
... Processing GENERAL#Negative
... Processing GENERAL#Neutral
... Processing SER&ACC#Positive
... Processing SER&ACC#Negative
... Processing SER&ACC#Neutral
... Processing OTHERS
                      precis

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#embedding + fasttext

In [ ]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.vec.gz

--2023-07-06 06:17:48--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.224.249.96, 13.224.249.94, 13.224.249.63, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.224.249.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1235219084 (1.1G) [binary/octet-stream]
Saving to: ‘cc.vi.300.vec.gz.1’

cc.vi.300.vec.gz.1  100%[===================>]   1.15G   181MB/s    in 8.6s    

2023-07-06 06:17:56 (137 MB/s) - ‘cc.vi.300.vec.gz.1’ saved [1235219084/1235219084]



In [ ]:
import gzip
import shutil
with gzip.open('cc.vi.300.vec.gz', 'rb') as f_in:
    with open('cc.vi.300.vec', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
EMBEDDING_FILE= '/content/cc.vi.300.vec'

In [ ]:
max_features=2489
maxlen=150
embed_size=300

In [ ]:
y_train = dd_train
y_test = dd_test

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(X_train))
word_index = tokenizer.word_index

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
!pip install keras_preprocessing

In [ ]:
from keras_preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print("create vector")

create vector


In [ ]:
embeddings_index = {}
with open(EMBEDDING_FILE) as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
embedding_dim = 300
embedding_matrix = np.zeros((max_features, embedding_dim))

In [ ]:
for word, i in word_index.items():
    if i >= max_features:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector[:embedding_dim]

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import os, re, csv, math, codecs

In [ ]:
#training params
batch_size = 32
epochs =20

#model parameters
num_filters = 64
embed_dim = 300
weight_decay = 1e-4

In [ ]:
!pip install tensorflow==2.4.1
!pip install keras==2.4.3

ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0)
ERROR: No matching distribution found for tensorflow==2.4.1


In [ ]:
!pip install --upgrade keras tensorflow

##cnn polaryti

In [ ]:
print("training CNN ...")
model = Sequential()
model.add(Embedding(max_features, embed_size,
          weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(31, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc',f1_m,precision_m, recall_m])
model.summary()

training CNN ...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 300)          746700    
                                                                 
 conv1d_2 (Conv1D)           (None, 150, 64)           134464    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 75, 64)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 75, 64)            28736     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 6

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [ ]:
#model training
hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=1)

Epoch 1/20
219/219 [==============================] - 15s 8ms/step - loss: 0.3648 - acc: 0.0402 - f1_m: 0.1216 - precision_m: 0.4018 - recall_m: 0.1010 - val_loss: 0.2644 - val_acc: 0.0359 - val_f1_m: 0.1390 - val_precision_m: 0.6989 - val_recall_m: 0.0778
Epoch 2/20
219/219 [==============================] - 1s 5ms/step - loss: 0.2687 - acc: 0.0866 - f1_m: 0.2833 - precision_m: 0.6443 - recall_m: 0.1876 - val_loss: 0.2321 - val_acc: 0.1682 - val_f1_m: 0.3981 - val_precision_m: 0.7160 - val_recall_m: 0.2771
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.2490 - acc: 0.1427 - f1_m: 0.3800 - precision_m: 0.6632 - recall_m: 0.2686 - val_loss: 0.2208 - val_acc: 0.1874 - val_f1_m: 0.4685 - val_precision_m: 0.6997 - val_recall_m: 0.3540
Epoch 4/20
219/219 [==============================] - 2s 7ms/step - loss: 0.2388 - acc: 0.1690 - f1_m: 0.4116 - precision_m: 0.6756 - recall_m: 0.2984 - val_loss: 0.2158 - val_acc: 0.1630 - val_f1_m: 0.4958 - val_precision_m: 0.701

In [ ]:
model.save ('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/UIT-ViSFD/CNN.h5')

In [ ]:
predictions = model.predict(X_test, batch_size=batch_size, verbose=1)

70/70 [==============================] - 0s 4ms/step


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 0s 5ms/step - loss: 0.1960 - acc: 0.2549 - f1_m: 0.5671 - precision_m: 0.7578 - recall_m: 0.4552
Test accuracy: [0.1960243135690689, 0.2549460530281067, 0.5671254396438599, 0.7578498721122742, 0.45521625876426697]


#BiRGU

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K

In [ ]:
inp = Input(shape=(maxlen, ))

x = Embedding(max_features, embed_size)(inp)
x = SpatialDropout1D(0.35)(x)

# Mạng GRU phức tạp
x = Bidirectional(GRU(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)

x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)


# Kết hợp đầu ra từ các lớp GRU
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = Concatenate()([avg_pool, max_pool])


x = Dense(31, activation="sigmoid")(conc)

# Xây dựng mô hình
model = Model(inputs=inp, outputs=x)

# Biên dịch mô hình
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
batch_size = 32
epochs = 20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/20
244/244 [==============================] - 353s 1s/step - loss: 0.2690 - acc: 0.0839 - f1_m: 0.2740 - precision_m: 0.5872 - recall_m: 0.1895
Epoch 2/20
244/244 [==============================] - 328s 1s/step - loss: 0.2051 - acc: 0.1978 - f1_m: 0.5257 - precision_m: 0.7305 - recall_m: 0.4141
Epoch 3/20
244/244 [==============================] - 311s 1s/step - loss: 0.1776 - acc: 0.2705 - f1_m: 0.6176 - precision_m: 0.7745 - recall_m: 0.5160
Epoch 4/20
244/244 [==============================] - 317s 1s/step - loss: 0.1592 - acc: 0.3029 - f1_m: 0.6711 - precision_m: 0.7956 - recall_m: 0.5827
Epoch 5/20
244/244 [==============================] - 326s 1s/step - loss: 0.1458 - acc: 0.3136 - f1_m: 0.7033 - precision_m: 0.8098 - recall_m: 0.6235
Epoch 6/20
244/244 [==============================] - 321s 1s/step - loss: 0.1343 - acc: 0.3176 - f1_m: 0.7298 - precision_m: 0.8223 - recall_m: 0.6576
Epoch 7/20
244/244 [==============================] - 322s 1s/step - loss: 0.1251 - acc:

In [ ]:
predictions = model.predict(X_test, batch_size=batch_size, verbose=1)

print(predictions)

70/70 [==============================] - 17s 225ms/step
[[9.74454045e-01 1.81822444e-03 1.97979665e-04 ... 1.13051157e-09
  3.89230649e-07 3.12786578e-05]
 [2.94536562e-03 8.45600292e-03 5.53694663e-05 ... 1.07121632e-04
  2.37663844e-04 5.04891723e-02]
 [4.74542467e-05 2.11521778e-02 4.00283752e-04 ... 2.69792799e-04
  8.99434555e-03 1.39068244e-02]
 ...
 [1.24647784e-07 3.66260224e-07 2.50886751e-06 ... 9.45080865e-06
  1.04633195e-03 3.00644673e-02]
 [8.31662398e-03 7.19576931e-08 7.94598745e-05 ... 1.08248496e-04
  1.75090078e-02 7.95848109e-03]
 [1.46486730e-06 1.57623361e-07 7.18321235e-06 ... 9.10108443e-03
  2.31353086e-04 2.02415571e-01]]


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 8s 95ms/step - loss: 0.2396 - acc: 0.3076 - f1_m: 0.6806 - precision_m: 0.7091 - recall_m: 0.6559
Test accuracy: [0.23958060145378113, 0.3075539469718933, 0.6806296110153198, 0.7091190218925476, 0.6559436917304993]


In [ ]:
model.save ('/content/drive/MyDrive/UIT-viFSD_me/BiGRU_68.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#BiGRU + pre



In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback


In [ ]:
inp = Input(shape=(maxlen, ))

x = Embedding(max_features, embed_size)(inp)
x = SpatialDropout1D(0.35)(x)

# Mạng GRU phức tạp
x = Bidirectional(GRU(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)

x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)


# Kết hợp đầu ra từ các lớp GRU
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = Concatenate()([avg_pool, max_pool])


x = Dense(31, activation="sigmoid")(conc)

# Xây dựng mô hình
model = Model(inputs=inp, outputs=x)

# Biên dịch mô hình
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
batch_size = 32
epochs = 20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)


Epoch 1/20
244/244 [==============================] - 395s 2s/step - loss: 0.2717 - acc: 0.0836 - f1_m: 0.2625 - precision_m: 0.6031 - recall_m: 0.1766
Epoch 2/20
244/244 [==============================] - 367s 2s/step - loss: 0.2054 - acc: 0.1700 - f1_m: 0.5252 - precision_m: 0.7413 - recall_m: 0.4113
Epoch 3/20
244/244 [==============================] - 363s 1s/step - loss: 0.1688 - acc: 0.2557 - f1_m: 0.6425 - precision_m: 0.7906 - recall_m: 0.5434
Epoch 4/20
244/244 [==============================] - 365s 1s/step - loss: 0.1480 - acc: 0.3071 - f1_m: 0.7021 - precision_m: 0.8173 - recall_m: 0.6174
Epoch 5/20
244/244 [==============================] - 366s 1s/step - loss: 0.1335 - acc: 0.3288 - f1_m: 0.7359 - precision_m: 0.8310 - recall_m: 0.6623
Epoch 6/20
244/244 [==============================] - 362s 1s/step - loss: 0.1219 - acc: 0.3373 - f1_m: 0.7621 - precision_m: 0.8446 - recall_m: 0.6959
Epoch 7/20
244/244 [==============================] - 340s 1s/step - loss: 0.1118 - acc:

In [ ]:
predictions = model.predict(X_test, batch_size=batch_size, verbose=1)

print(predictions)

70/70 [==============================] - 24s 315ms/step
[[9.42162216e-01 6.77466323e-06 1.52522800e-04 ... 2.06600433e-07
  8.52087469e-05 6.19723811e-04]
 [4.31951321e-06 5.66727817e-01 1.04891424e-05 ... 6.09850613e-05
  1.50483145e-04 3.16402479e-03]
 [1.01814384e-03 2.50837742e-03 1.18073796e-04 ... 4.79059236e-06
  1.76043448e-03 2.38286898e-01]
 ...
 [2.67420518e-07 8.05101081e-07 3.81323503e-07 ... 4.40132317e-05
  1.01395381e-04 1.04604354e-02]
 [7.39164170e-05 5.71321435e-09 6.06526055e-06 ... 8.06000244e-05
  1.73594508e-05 3.90336327e-02]
 [2.51541696e-05 7.95210781e-07 2.88532055e-06 ... 1.71528900e-05
  1.42682251e-03 2.32584607e-02]]


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 11s 136ms/step - loss: 0.2008 - acc: 0.3156 - f1_m: 0.7283 - precision_m: 0.7580 - recall_m: 0.7020
Test accuracy: [0.2008216828107834, 0.3156474828720093, 0.7282615900039673, 0.7579637169837952, 0.7019937634468079]


In [ ]:
model.save ('/content/drive/MyDrive/UIT-viFSD_me/phobert_BiGRU.h5')

#lstm


In [ ]:
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15)(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(31, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150)]        0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 150, 300)     746700      ['input_2[0][0]']                
                                                                                                  
 spatial_dropout1d_1 (SpatialDr  (None, 150, 300)    0           ['embedding_3[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 lstm (LSTM)                    (None, 150, 128)     219648      ['spatial_dropout1d_1[0][0]

In [ ]:
batch_size = 32
epochs =20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(X_test, batch_size=batch_size, verbose=1)

print(predictions)

Epoch 1/20
244/244 [==============================] - 202s 807ms/step - loss: 0.2834 - acc: 0.0866 - f1_m: 0.2330 - precision_m: 0.5262 - recall_m: 0.1634
Epoch 2/20
244/244 [==============================] - 181s 742ms/step - loss: 0.2134 - acc: 0.2096 - f1_m: 0.5044 - precision_m: 0.7252 - recall_m: 0.3904
Epoch 3/20
244/244 [==============================] - 180s 740ms/step - loss: 0.1839 - acc: 0.2709 - f1_m: 0.6003 - precision_m: 0.7635 - recall_m: 0.4969
Epoch 4/20
244/244 [==============================] - 178s 730ms/step - loss: 0.1649 - acc: 0.2994 - f1_m: 0.6564 - precision_m: 0.7904 - recall_m: 0.5633
Epoch 5/20
244/244 [==============================] - 179s 733ms/step - loss: 0.1511 - acc: 0.3260 - f1_m: 0.6904 - precision_m: 0.8088 - recall_m: 0.6039
Epoch 6/20
244/244 [==============================] - 179s 734ms/step - loss: 0.1403 - acc: 0.3351 - f1_m: 0.7169 - precision_m: 0.8191 - recall_m: 0.6387
Epoch 7/20
244/244 [==============================] - 174s 712ms/step 

In [ ]:
model.save ('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/UIT-ViSFD/LSTM.h5')

In [ ]:
pre=pd.DataFrame(predictions)
pre.columns=label

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 3s 43ms/step - loss: 0.2093 - acc: 0.2918 - f1_m: 0.6782 - precision_m: 0.7227 - recall_m: 0.6405
Test accuracy: [0.20929060876369476, 0.2918165326118469, 0.6781962513923645, 0.722746729850769, 0.640539288520813]


In [ ]:
pre

,SCREEN#Positive,SCREEN#Negative,SCREEN#Neutral,CAMERA#Positive,CAMERA#Negative,CAMERA#Neutral,FEATURES#Positive,FEATURES#Negative,FEATURES#Neutral,BATTERY#Positive,...,PRICE#Positive,PRICE#Negative,PRICE#Neutral,GENERAL#Positive,GENERAL#Negative,GENERAL#Neutral,SER&ACC#Positive,SER&ACC#Negative,SER&ACC#Neutral,OTHERS
0,0.780966,0.002742,0.000777,0.000630,0.000004,0.001045,0.986993,0.000584,0.258115,0.953677,...,0.000370,1.675197e-05,0.002783,0.677906,0.000003,0.003545,0.000084,0.000063,0.000083,0.016107
1,0.000421,0.016447,0.000047,0.000189,0.000056,0.000012,0.025000,0.021757,0.020761,0.001083,...,0.000001,5.753789e-06,0.000309,0.020047,0.000023,0.003445,0.308471,0.004041,0.009682,0.729370
2,0.000003,0.000094,0.000059,0.000261,0.005138,0.000012,0.000311,0.000039,0.000115,0.000119,...,0.000985,5.615622e-04,0.001812,0.769604,0.001840,0.000227,0.999836,0.000862,0.013582,0.087530
3,0.002070,0.675044,0.000203,0.000096,0.003308,0.000128,0.007276,0.090512,0.041603,0.001146,...,0.000011,8.461452e-04,0.000195,0.047968,0.004988,0.006574,0.000016,0.000731,0.000085,0.159052
4,0.584193,0.086755,0.052760,0.000519,0.000020,0.000927,0.008292,0.000036,0.000191,0.024225,...,0.000532,2.655961e-01,0.951139,0.117601,0.044387,0.002715,0.000071,0.019569,0.001812,0.307863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219,0.000498,0.000010,0.000014,0.000124,0.000002,0.000009,0.187421,0.470672,0.001411,0.000027,...,0.001393,6.094281e-04,0.000013,0.012500,0.000719,0.000794,0.355898,0.000268,0.000101,0.027444
2220,0.005387,0.013170,0.001823,0.000397,0.001800,0.017062,0.000717,0.005638,0.001963,0.000336,...,0.000790,2.258047e-03,0.000589,0.049872,0.003896,0.204192,0.000081,0.000414,0.000613,0.042261
2221,0.000014,0.000988,0.000069,0.000186,0.000036,0.000035,0.000649,0.003651,0.000380,0.000128,...,0.005033,1.248889e-04,0.000322,0.204231,0.001637,0.001451,0.023221,0.002750,0.000895,0.025311
2222,0.000520,0.000003,0.000013,0.938562,0.001102,0.016421,0.717132,0.000291,0.000365,0.836907,...,0.000197,3.199929e-07,0.000111,0.361033,0.000013,0.000224,0.999345,0.000023,0.000788,0.025356


In [ ]:
c_pre=pre.copy()

In [ ]:
for i in range(0,c_pre.shape[0]):
  for j in range(0,len(label)):
    if c_pre[label[j]][i]>=0.5:
      c_pre[label[j]][i]=1
    else:
      c_pre[label[j]][i]=0

print(classification_report(y_test, c_pre, target_names=label))

                      precision    recall  f1-score   support

     SCREEN#Positive       0.79      0.65      0.71       136
     SCREEN#Negative       0.59      0.57      0.58       116
      SCREEN#Neutral       0.00      0.00      0.00        17
     CAMERA#Positive       0.87      0.81      0.84       346
     CAMERA#Negative       0.76      0.61      0.68       171
      CAMERA#Neutral       0.53      0.39      0.45        71
   FEATURES#Positive       0.55      0.64      0.59       200
   FEATURES#Negative       0.78      0.63      0.70       459
    FEATURES#Neutral       0.33      0.08      0.12        52
    BATTERY#Positive       0.85      0.80      0.82       554
    BATTERY#Negative       0.76      0.79      0.78       368
     BATTERY#Neutral       0.32      0.15      0.21        92
PERFORMANCE#Positive       0.80      0.77      0.79       602
PERFORMANCE#Negative       0.73      0.65      0.69       454
 PERFORMANCE#Neutral       0.30      0.20      0.24       116
    STO

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#BiLSTM

In [ ]:
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(31, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
batch_size = 32
epochs =20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(X_test, batch_size=batch_size, verbose=1)

print(predictions)

Epoch 1/20
244/244 [==============================] - 407s 2s/step - loss: 0.2701 - acc: 0.1053 - f1_m: 0.2821 - precision_m: 0.5831 - recall_m: 0.2050
Epoch 2/20
244/244 [==============================] - 353s 1s/step - loss: 0.1963 - acc: 0.2404 - f1_m: 0.5592 - precision_m: 0.7507 - recall_m: 0.4488
Epoch 3/20
244/244 [==============================] - 349s 1s/step - loss: 0.1688 - acc: 0.2937 - f1_m: 0.6420 - precision_m: 0.7820 - recall_m: 0.5468
Epoch 4/20
244/244 [==============================] - 349s 1s/step - loss: 0.1519 - acc: 0.3134 - f1_m: 0.6842 - precision_m: 0.8018 - recall_m: 0.5987
Epoch 5/20
244/244 [==============================] - 348s 1s/step - loss: 0.1394 - acc: 0.3292 - f1_m: 0.7188 - precision_m: 0.8165 - recall_m: 0.6435
Epoch 6/20
244/244 [==============================] - 361s 1s/step - loss: 0.1290 - acc: 0.3406 - f1_m: 0.7411 - precision_m: 0.8275 - recall_m: 0.6727
Epoch 7/20
244/244 [==============================] - 347s 1s/step - loss: 0.1196 - acc:

In [ ]:
model.save ('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/UIT-ViSFD/bilstm.h5')

In [ ]:
pre=pd.DataFrame(predictions)
pre.columns=label

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 10s 120ms/step - loss: 0.2466 - acc: 0.2878 - f1_m: 0.6832 - precision_m: 0.7133 - recall_m: 0.6566
Test accuracy: [0.24657152593135834, 0.28776979446411133, 0.6831986308097839, 0.7132704854011536, 0.6565975546836853]


In [ ]:
c_pre=pre.copy()

In [ ]:
c_pre

,SCREEN#Positive,SCREEN#Negative,SCREEN#Neutral,CAMERA#Positive,CAMERA#Negative,CAMERA#Neutral,FEATURES#Positive,FEATURES#Negative,FEATURES#Neutral,BATTERY#Positive,...,PRICE#Positive,PRICE#Negative,PRICE#Neutral,GENERAL#Positive,GENERAL#Negative,GENERAL#Neutral,SER&ACC#Positive,SER&ACC#Negative,SER&ACC#Neutral,OTHERS
0,0.999709,1.396474e-05,6.598023e-04,5.939688e-04,3.649387e-08,2.829044e-06,0.998396,0.003194,0.000194,0.999057,...,7.969548e-06,0.000129,0.000040,0.999934,2.442429e-07,1.457499e-08,0.000006,1.343879e-05,0.000003,0.000232
1,0.002293,7.304838e-01,2.554552e-06,3.257334e-07,5.260723e-04,1.921382e-06,0.005774,0.939195,0.014925,0.000009,...,2.591101e-09,0.000002,0.001678,0.000071,4.870849e-01,6.293978e-03,0.051367,2.497640e-02,0.021177,0.028505
2,0.003662,6.880208e-03,1.506535e-05,3.339217e-06,2.568313e-05,1.903624e-06,0.000009,0.000004,0.000002,0.000010,...,2.019252e-06,0.000005,0.000020,0.034609,5.155983e-04,2.685668e-04,0.999960,2.210593e-05,0.006729,0.035604
3,0.000041,7.233349e-02,6.556761e-04,1.024083e-05,5.316553e-06,1.868746e-05,0.002228,0.592137,0.101924,0.000025,...,3.785683e-07,0.000081,0.000040,0.083608,8.686893e-03,1.746645e-03,0.000004,4.361975e-07,0.000576,0.384203
4,0.365051,2.287787e-03,1.209269e-02,1.242027e-04,8.094037e-06,2.690757e-05,0.003836,0.000070,0.000192,0.000369,...,3.735439e-07,0.052013,0.999934,0.151073,2.769658e-02,5.787590e-03,0.000031,8.269659e-02,0.000589,0.244517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219,0.001341,5.597917e-08,3.038458e-07,1.833082e-03,2.852527e-06,1.818450e-03,0.967251,0.733946,0.222887,0.005024,...,1.143828e-04,0.000002,0.000051,0.000283,1.508103e-03,5.169203e-05,0.046078,8.480621e-04,0.000702,0.000060
2220,0.000007,2.003408e-02,5.101344e-04,6.633141e-04,1.549396e-02,2.986021e-03,0.000041,0.000585,0.006839,0.000175,...,4.072354e-05,0.000256,0.000011,0.001607,3.582772e-03,9.707218e-01,0.000001,7.571694e-06,0.000013,0.001003
2221,0.000028,2.801760e-04,1.078236e-05,8.719340e-05,5.128274e-08,1.044442e-06,0.000059,0.000062,0.000031,0.000017,...,1.680355e-05,0.000008,0.000006,0.161729,2.356015e-04,4.459704e-03,0.026837,6.273274e-04,0.000169,0.110068
2222,0.000362,1.666003e-07,1.235052e-05,9.962841e-01,7.538566e-04,6.716461e-03,0.666190,0.000013,0.000041,0.999942,...,1.548559e-03,0.000002,0.000087,0.184222,5.051742e-05,3.695758e-05,0.992700,6.516079e-04,0.000680,0.003094


In [ ]:
for i in range(0,c_pre.shape[0]):
  for j in range(0,len(label)):
    if c_pre[label[j]][i]>=0.5:
      c_pre[label[j]][i]=1
    else:
      c_pre[label[j]][i]=0

#biLSTM + pre



In [ ]:
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(31, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
batch_size = 32
epochs =20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(X_test, batch_size=batch_size, verbose=1)

print(predictions)

Epoch 1/20
244/244 [==============================] - 420s 2s/step - loss: 0.2668 - acc: 0.0976 - f1_m: 0.2875 - precision_m: 0.5923 - recall_m: 0.2070
Epoch 2/20
244/244 [==============================] - 359s 1s/step - loss: 0.1927 - acc: 0.2244 - f1_m: 0.5731 - precision_m: 0.7545 - recall_m: 0.4650
Epoch 3/20
244/244 [==============================] - 353s 1s/step - loss: 0.1635 - acc: 0.2695 - f1_m: 0.6600 - precision_m: 0.7965 - recall_m: 0.5661
Epoch 4/20
244/244 [==============================] - 353s 1s/step - loss: 0.1434 - acc: 0.2996 - f1_m: 0.7110 - precision_m: 0.8187 - recall_m: 0.6302
Epoch 5/20
244/244 [==============================] - 351s 1s/step - loss: 0.1287 - acc: 0.3109 - f1_m: 0.7482 - precision_m: 0.8364 - recall_m: 0.6785
Epoch 6/20
244/244 [==============================] - 343s 1s/step - loss: 0.1177 - acc: 0.3266 - f1_m: 0.7720 - precision_m: 0.8501 - recall_m: 0.7084
Epoch 7/20
244/244 [==============================] - 343s 1s/step - loss: 0.1080 - acc:

In [ ]:
pre=pd.DataFrame(predictions)
pre.columns=label

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 7s 90ms/step - loss: 0.2120 - acc: 0.3435 - f1_m: 0.7269 - precision_m: 0.7501 - recall_m: 0.7062
Test accuracy: [0.21204328536987305, 0.3435251712799072, 0.7269288897514343, 0.7501040697097778, 0.7062495350837708]


In [ ]:
model.save ('/content/drive/MyDrive/UIT-viFSD_me/phobert_BiLSTM.h5')

In [ ]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

In [ ]:
inp = Input(shape=(maxlen,))
z = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
x1 = SpatialDropout1D(0.35)(z)

x = Bidirectional(GRU(128, return_sequences = True, dropout=0.15, recurrent_dropout=0.15))(x1)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer ='glorot_uniform')(x)

y = Bidirectional(LSTM(128, return_sequences = True, dropout=0.15, recurrent_dropout=0.15))(x1)
y = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = 'glorot_uniform')(y)

avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)

avg_pool2 = GlobalAveragePooling1D()(y)
max_pool2 = GlobalMaxPooling1D()(y)

x = concatenate([avg_pool1, max_pool1, avg_pool2, max_pool2])

out = Dense(31, activation = "sigmoid")(x)
model = Model(inputs = inp, outputs = out)
model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=['acc',f1_m,precision_m, recall_m])


In [ ]:
batch_size = 32
epochs = 20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/20
244/244 [==============================] - 593s 2s/step - loss: 0.2696 - acc: 0.1205 - f1_m: 0.2841 - precision_m: 0.6346 - recall_m: 0.2007
Epoch 2/20
244/244 [==============================] - 562s 2s/step - loss: 0.2090 - acc: 0.2162 - f1_m: 0.5154 - precision_m: 0.7312 - recall_m: 0.4017
Epoch 3/20
244/244 [==============================] - 561s 2s/step - loss: 0.1927 - acc: 0.2421 - f1_m: 0.5711 - precision_m: 0.7533 - recall_m: 0.4623
Epoch 4/20
244/244 [==============================] - 570s 2s/step - loss: 0.1824 - acc: 0.2624 - f1_m: 0.6021 - precision_m: 0.7647 - recall_m: 0.4987
Epoch 5/20
244/244 [==============================] - 558s 2s/step - loss: 0.1734 - acc: 0.2719 - f1_m: 0.6276 - precision_m: 0.7750 - recall_m: 0.5296
Epoch 6/20
244/244 [==============================] - 558s 2s/step - loss: 0.1657 - acc: 0.2865 - f1_m: 0.6481 - precision_m: 0.7883 - recall_m: 0.5524
Epoch 7/20
244/244 [==============================] - 556s 2s/step - loss: 0.1597 - acc:

In [ ]:
pred = model.predict(X_test, batch_size = 32, verbose = 1)
print(pred)

70/70 [==============================] - 23s 287ms/step
[[7.2324353e-01 6.1352656e-04 4.6715569e-03 ... 3.7920178e-04
  1.4639364e-03 2.2429163e-02]
 [6.2140990e-02 9.0356199e-03 8.3081465e-04 ... 4.3545519e-03
  2.9306265e-03 1.2041051e-01]
 [5.1588367e-04 4.5398669e-03 1.0506931e-03 ... 3.5809956e-02
  1.7546393e-02 2.0061916e-01]
 ...
 [2.6606317e-03 3.4163778e-03 1.5228083e-04 ... 1.3636565e-03
  5.4915744e-04 4.7549602e-02]
 [2.0475058e-02 9.2209457e-06 7.5901502e-05 ... 1.6456947e-03
  6.5794350e-03 1.4469196e-01]
 [1.4737135e-03 1.0664180e-03 5.3731064e-06 ... 1.5226604e-02
  1.8948948e-02 4.7977728e-01]]


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 14s 172ms/step - loss: 0.1748 - acc: 0.3138 - f1_m: 0.6810 - precision_m: 0.7372 - recall_m: 0.6341
Test accuracy: [0.17482487857341766, 0.31384891271591187, 0.6810162663459778, 0.7371849417686462, 0.6340978145599365]


In [ ]:
model.save ('/content/drive/MyDrive/KhóaLuậnTốtNghiệp/UIT-ViSFD/BiGRU_BiLSTM.h5')

#CNN + LSTM

In [ ]:
model = Sequential()
inp = Input(shape=(maxlen,))
model.add(Embedding(max_features, embed_size,
          weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', input_shape=(178, 1), padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))
model.add(Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform'))
model.add(Flatten())
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(31, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 300)          746700    
                                                                 
 conv1d_1 (Conv1D)           (None, 150, 64)           134464    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 75, 64)           0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 75, 64)            28736     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 37, 64)           0         
 1D)                                                             
                                                                 
 dropout (Dropout)           (None, 37, 64)            0

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=20, batch_size=32, verbose=2)

Epoch 1/20
244/244 - 52s - loss: 0.3069 - acc: 0.0804 - f1_m: 0.2185 - precision_m: 0.4967 - recall_m: 0.1640 - val_loss: 0.2484 - val_acc: 0.1016 - val_f1_m: 0.3323 - val_precision_m: 0.6545 - val_recall_m: 0.2244 - 52s/epoch - 215ms/step
Epoch 2/20
244/244 - 48s - loss: 0.2473 - acc: 0.0999 - f1_m: 0.3435 - precision_m: 0.6504 - recall_m: 0.2383 - val_loss: 0.2356 - val_acc: 0.1277 - val_f1_m: 0.3526 - val_precision_m: 0.6719 - val_recall_m: 0.2415 - 48s/epoch - 198ms/step
Epoch 3/20
244/244 - 47s - loss: 0.2343 - acc: 0.1065 - f1_m: 0.3992 - precision_m: 0.6609 - recall_m: 0.2906 - val_loss: 0.2261 - val_acc: 0.1174 - val_f1_m: 0.4405 - val_precision_m: 0.6430 - val_recall_m: 0.3378 - 47s/epoch - 194ms/step
Epoch 4/20
244/244 - 47s - loss: 0.2246 - acc: 0.1142 - f1_m: 0.4433 - precision_m: 0.6698 - recall_m: 0.3349 - val_loss: 0.2197 - val_acc: 0.1156 - val_f1_m: 0.4186 - val_precision_m: 0.6944 - val_recall_m: 0.3020 - 47s/epoch - 192ms/step
Epoch 5/20
244/244 - 49s - loss: 0.2161 

In [ ]:
print("Evaluate Model Performance on Test set")
result = model.predict(X_test, batch_size = 32, verbose = 2)

Evaluate Model Performance on Test set
70/70 - 2s - 2s/epoch - 25ms/step


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 1s 14ms/step - loss: 0.2054 - acc: 0.2698 - f1_m: 0.6232 - precision_m: 0.7308 - recall_m: 0.5449
Test accuracy: [0.20537225902080536, 0.26978418231010437, 0.6232392191886902, 0.7308080196380615, 0.5449463129043579]


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score)

70/70 [==============================] - 1s 20ms/step - loss: 0.2046 - acc: 0.2419 - f1_m: 0.5783 - precision_m: 0.7251 - recall_m: 0.4822
Test accuracy: [0.2046307474374771, 0.24190647900104523, 0.5783098936080933, 0.7251214981079102, 0.48220738768577576]
